<h2>Project 4: Empirical Risk Minimization</h2>

<blockquote>
    <center>
    <img src="./spam.jpeg" width="200px" />
    </center>
      <p><cite><center>"One person's spam is another person's dinner."<br>
       -- ancient German wisdon
      </center></cite></p>
</blockquote>

<h3>Introduction</h3>

<p>In this project you will be building an email spam filter.</p>

<strong>How to submit:</strong> You can submit your code using the red <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.
                </p>

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/iyag4nk2rsxsv">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.  </p>

<p><strong>Efficiency:</strong> You may run into the autograder time limit for this project (especially if you are competing in the competition). A few tips:</p>

<ol>
<li>Avoid for loops in the loss functions.</li>
<li>Store large matrix multiplications in temporary variables if they need to be reused.</li>
</ol>

<h3>Computing derivatives</h3>

<p>  Before you dive into the programming part of this assignment you will need to derive the gradients for several loss functions. 
    <b>You do not need to hand this part in for Project 4 but you will need to hand it in for Homework 3.</b> 
</p>

<p>   Derive the gradient function for each of the following loss functions with respect to the weight vector $w$. Write down the gradient update (with stepsize $c$). <br>
(Note that:    $\|w\|_2^2=w^\top w$ and  $\lambda$ is a  non-negative constant.)
</p>

<ol>
    <li> Ridge Regression: ${\cal L}(w)=\frac{1}{n}\sum_{i=1}^n (w^\top x_i-y_i)^2+\lambda \|w\|_2^2$ </li>
    <li> Logistic Regression: ($y_i\in\{+1,-1\}$): ${\cal L}(w)=\sum_{i=1}^n \log(1+\exp{(-y_i w^\top x_i)})$ </li>
    <li> Hinge loss: ($y_i\in\{+1,-1\}$): ${\cal L}(w)=\sum_{i=1}^n \max \left(1-y_i(w^\top x_i+b),0\right)+\lambda \|w\|_2^2$ </li>
</ol>  

<h3>Building an email spam filter</h3>
<p> You will now implement ridge loss and the Adagrad algorithm.
   
The function below loads in pre-processed email data, where emails are represented as bag-of-words vectors.
</p>



In [ ]:
# You should have run this code before, but just in case
try
    using MAT
catch
    Pkg.add("MAT")
    Pkg.build("MAT")
end
try
    using PyPlot
catch
    Pkg.add("PyPlot")
    Pkg.build("PyPlot")
end

In [ ]:
function extractfeaturesnaive(path,B)
    # tokenize the email and hashes the symbols into a vector
    femail=open(path);
    v=zeros(B,1);              # initialize all-zeros feature vector
    email=readstring(femail);
    tokens=split(email);       # breaks for non-ascii characters
    for token=split(email);
        v[mod(hash(token),B)+1]=1;
    end;
    close(femail);
    return(v);
end;

function loadspamdata(extractfeatures;B=512,path="../resource/lib/public/data_train/")
    print(path)

    f=open(path*"index");
    allemails=filter(s -> issubset(" ",s),split(readstring(f),"\n"));
    xs=zeros(length(allemails),B);
    ys=zeros(length(allemails),1);
    for (i,line) ∈ enumerate(allemails)
        label,filename=split(line," ")
        ys[i]=(label=="spam").*2.0-1.0; # make labels +1 for "spam" and -1 for "ham"
        xs[i,:]=extractfeatures(path*filename, B);
    end;
    close(f);
    @printf("Loaded %i input emails.\n",length(ys));
    return(xs,ys);
end;

X,Y=loadspamdata(extractfeaturesnaive);
print(size(X));

This is your training set. To evaluate your algorithm you should split it off into a validation set.

In [ ]:
# Split data into training and validation
n,d=size(X);
cutoff=Int(ceil(0.8*n));
itr=1:cutoff;    # indices of training samples
ite=cutoff.+1:n; # indices of testing samples
xTr=X[itr,:];
yTr=Y[itr];
xTv=X[ite,:];
yTv=Y[ite];

<p>This should generate a training data set <code>xTr</code>, <code>yTr</code> and a validation set <code>xTv</code>, <code>yTv</code> for you. (You can check what variables is in your environment with the <code>whos()</code> command.) 
</p>

<p>It is now time to implement your classifiers. We will always use the Adagrad gradient descent algorithm, but with various loss functions. 
First implement the function <code>ridge</code> which computes the ridge regression loss and gradient for a particular data set <code>xTr</code>, <code>yTr</code> and a weight vector <code>w</code>. Make sure you don't forget to incorporate your regularization constant $\lambda$. </p>


In [ ]:
#<GRADED>
function ridge(w,xTr,yTr,lmbda)
    #
    # INPUT:
    # w     : d   dimensional weight vector
    # xTr   : nxd dimensional matrix (each column is an input vector)
    # yTr   : n   dimensional vector (each entry is a label)
    # lmbda : regression constant (scalar)
    #
    # OUTPUTS:
    # loss     : the total loss obtained with w on xTr and yTr (scalar)
    # gradient : d dimensional gradient at w
    #
    
    n,d=size(xTr);
    
    loss = 0;
    gradient = zeros(d);
    
    ## fill in code here

    return(loss[1],gradient[:]);
end;
#</GRADED>

<p>An  alternative to  deriving the gradient analytically is to estimate it numerically. This is very slow, but it is a convenient  way to check your code for correctness.  The following function  uses numerical differentiation to evaluate the correctness of ridge.  If your code is correct, the norm difference between the two should be very small (smaller than $10^{-5}$). 
Keep in mind that this only checks if the gradient corresponds to the loss, but not if the loss is correct. The function also plots an image of the gradient values (blue) and their estimates (red). If everything is correct, these two should be right on top of each other.
</p>

In [ ]:
function numericalgradient(fun,w,e);
    d=length(w);                   # get dimensionality
    dh=zeros(d,1);                 # initialize numerical derivative
    for i=1:d                      # go through dimensions
        nw=w;                      # copy the weight vector
        nw[i]+=e;                  # perturb dimension i
        l1,temp=fun(nw);           # compute loss
        nw[i]-=2*e;                # perturb dimension i again
        l2,temp=fun(nw);           # compute loss
        dh[i]=(l1[1]-l2[1])/(2*e); # the gradient is the slope of the loss
    end;
    return(dh[:]);
end;

function checkgrad(fun,w,e);
    loss,dy=fun(w);                # evaluate symbolic gradient from fun()    
    dh=numericalgradient(fun,w,e); # estimate gradient numerically from fun()
    
    ii=sortperm(dy);
    ii=1:length(dy);
    plot(dh[ii],"bo");
    hold(true);
    plot(dy[ii],"r.");
    xlabel("Dimension");
    ylabel("Gradient value");
    legend(["numeric","symbolic"]);

    return(norm(dh-dy)/norm(dh+dy)); # return the norm of the difference scaled by the norm of the sum
end;

lmbda=0.1;     # set lambda arbitrarily
d=size(xTr,2); # dimensionality of the input
w=rand(d,1);   # evaluate loss on random vector
ratio=checkgrad(w->ridge(w,xTr,yTr,lmbda),w,1e-05); # the a->fun(a) is an inline way to define a function with only a single argument.
print("The norm ratio is $ratio.")

<p>Implement the function <code>adagrad</code> which performs adaptive gradient descent. 
Make sure to include the tolerance variable to stop early if the norm of the gradient is less than the tolerance value (you can use the function <code>norm(x)</code>). When the norm of the gradient is tiny it means that you have arrived at a minimum.  <br>
The first parameter of <code>adagrad</code> is a function which takes a weight vector and returns loss and gradient.
</p>                

In [ ]:
#<GRADED>
function adagrad(func,w,alpha,maxiter;delta=1e-02)
    #
    # INPUT:
    # func    : function to minimize
    #           (loss, gradient = func(w))
    # w       : d dimensional initial weight vector 
    # alpha   : initial gradient descent stepsize (scalar)
    # maxiter : maximum amount of iterations (scalar)
    # delta   : if norm(gradient)<delta, it quits (scalar)
    #
    # OUTPUTS:
    # 
    # w      : d dimensional final weight vector
    # losses : vector containing loss at each iteration
    #
    
    losses=zeros(maxiter);
    epsilon=1e-06; # Adagrad epsilon term
    
    ## fill in code here

    return(w[:],losses);
end;
#</GRADED>

In [ ]:
w,losses=adagrad(w->ridge(w,xTr,yTr,lmbda),rand(size(xTr,2)),1,1000);

using PyPlot
figure()
semilogy(losses,"r-");
xlabel("gradient updates");
ylabel("loss");
title("Adagrad convergence");
@printf("Final loss: %e",losses[end]);

<p> Write the (almost trivial) function <code>linclassify</code> which returns the predictions for a vector <code>w</code> and a data set <code>xTv</code>. (You can take it from a previous project.)</p>

<p>After this you can check your training and validation accuracy by running the cell below.</p>

In [ ]:
#<GRADED>

function linclassify(w,xTr);
    preds = zeros(size(xTr, 1));
    
    ## fill in code here
    
    return preds;
end;
#</GRADED>
    
# evaluate training accuracy
preds=linclassify(w,xTr);

trainingacc=mean(preds.==yTr);
# evaluate testing accuracy
preds=linclassify(w,xTv);
validationacc=mean(preds.==yTv);
@printf("Training accuracy %2.2f%%\nValidation accuracy %2.2f%%\n",trainingacc*100,validationacc*100);

<p>Now implement the two other loss functions, <code>logistic</code> and <code>hinge</code>. Start off with <code>logistic</code>:</p>

In [ ]:
#<GRADED>
function logistic(w,xTr,yTr)
    #
    # INPUT:
    # w     : d   dimensional weight vector
    # xTr   : nxd dimensional matrix (each column is an input vector)
    # yTr   : n   dimensional vector (each entry is a label)
    #
    # OUTPUTS:
    # loss     : the total loss obtained with w on xTr and yTr (scalar)
    # gradient : d dimensional gradient at w
    #
    
    n,d=size(xTr);
    
    loss = 0;
    gradient = zeros(d);
    
    ## fill in code here

    return(loss[1],gradient[:]);
end;
#</GRADED>

<p>You can use the two cells below to test how well this loss function performs.</p>

In [ ]:
# Gradient sanity check
d=size(xTr,2);
w=rand(d,1);
ratio=checkgrad(w->logistic(w,xTr,yTr),w,1e-05);
print("The norm ratio is $ratio.")

In [ ]:
# Train using logistic loss
w,losses=adagrad(w->logistic(w,xTr,yTr),rand(size(xTr,2)),1,1000);

# evaluate training accuracy
preds=linclassify(w,xTr);
trainingacc=mean(preds.==yTr);
# evaluate testing accuracy
preds=linclassify(w,xTv);
validationacc=mean(preds.==yTv);
@printf("Training accuracy %2.2f%%\nValidation accuracy %2.2f%%\n",trainingacc*100,validationacc*100);

<p>Now implement <code>hinge</code>:</p>

In [ ]:
#<GRADED>
function hinge(w,xTr,yTr,lmbda)
    #
    # INPUT:
    # w     : d   dimensional weight vector
    # xTr   : nxd dimensional matrix (each column is an input vector)
    # yTr   : n   dimensional vector (each entry is a label)
    # lmbda : regression constant (scalar)
    #
    # OUTPUTS:
    # loss     : the total loss obtained with w on xTr and yTr (scalar)
    # gradient : d dimensional gradient at w
    #
    
    n,d=size(xTr);
    
    loss = 0;
    gradient = zeros(d);
    
    ## fill in code here
    
    return(loss[1],gradient[:]);
end;
#</GRADED>

<p>You can use the two cells below to test how well this loss function performs.</p>

In [ ]:
# Gradient sanity check
lmbda=0.1;
d=size(xTr,2);
w=rand(d,1);
ratio=checkgrad(w->hinge(w,xTr,yTr,lmbda),w,1e-05);
print("The norm ratio is $ratio.")

In [ ]:
# Train using hinge loss
w,losses=adagrad(w->hinge(w,xTr,yTr,lmbda),rand(size(xTr,2)),1,1000);

# evaluate training accuracy
preds=linclassify(w,xTr);
trainingacc=mean(preds.==yTr);
# evaluate testing accuracy
preds=linclassify(w,xTv);
validationacc=mean(preds.==yTv);
@printf("Training accuracy %2.2f%%\nValidation accuracy %2.2f%%\n",trainingacc*100,validationacc*100);

<h3>Competition <b>(Optional)</b></h3>

<p>The competition for this assignment is split into two components:</p>

<ol>
<li><b>Feature Extraction</b>:
Modify the function <code>extractfeaturescomp</code>.
This function takes in a file path <code>path</code> and
a feature dimension <code>B</code> and should output a feature vector of dimension <code>B</code>.
The autograder will pass in a file path pointing to a file that contains an email,
and set <code>B</code> = <code>feature_dimension</code>.
We provide <code>extractfeaturesnaive</code> as an example.
</li>
<li><b>Model Training</b>:
Modify the function <code>trainspamfiltercomp</code>.
This function takes in training data <code>xTr</code> and training labels <code>yTr</code> and
should output a weight vector <code>w</code> for linear classification.
</li>
</ol>

<p>Your model will be trained on the same training set above (loaded by <code>loadspamdata</code>), but we will test its accuracy on a secret dataset of emails.</p>

In [ ]:
#<GRADED>
# dimensionality of feature vectors you want to use for the competition
feature_dimension = 512;

function extractfeaturescomp(path, B)
    #
    # INPUT:
    # path : file path of email
    # B    : dimensionality of feature vector
    #
    # OUTPUTS:
    # x    : B dimensional vector
    
    #x = extractfeaturesnaive(path, B);
      
    # tokenize the email and hashes the symbols into a vector
    femail=open(path);
    x=zeros(B,1);              # initialize all-zeros feature vector
    email=readstring(femail);
    tokens=split(email);       # breaks for non-ascii characters
    for token=split(email);
        x[mod(hash(token),B)+1]=1;
    end;
    close(femail);
    return(x);
end;
#</GRADED>

In [ ]:
#<GRADED>
function trainspamfiltercomp(xTr,yTr)
    #
    # INPUT:
    # xTr : nxd dimensional matrix (each column is an input vector)
    # yTr : d   dimensional vector (each entry is a label)
    #
    # OUTPUTS:
    # w : d dimensional vector for linear classification
    #
    
    w = rand(size(xTr, 2));

    return w[:];
end;
#</GRADED>